In [30]:
import glob
from langchain_ollama import OllamaLLM
import numpy as np
import pandas as pd
import ast
from difflib import SequenceMatcher

stigmatizingLanguage = []

def askLlama(prompt, OGsentence):
    print(prompt)
    result = model.invoke(input=prompt)
    print(result)
    a = ast.literal_eval(result)
    highestStringSimilarity = 0
    string = ""
    for i in list(a.values()):
        print(i)
        print(SequenceMatcher(None, i, OGsentence).ratio())
        if SequenceMatcher(None, i, OGsentence).ratio() > highestStringSimilarity and SequenceMatcher(None, i, OGsentence).ratio() < 1:
            highestStringSimilarity = SequenceMatcher(None, i, OGsentence).ratio()
            string = i
    return string

# Creates a 2D array with first element as the note number with the bad stuff and the second element as the identified word
with open("listOfAllStigmatizingLanguage.txt") as f:
    for i in f.read().split("\n"):
        stigmatizingLanguage.append([list(ast.literal_eval(i).keys())[0], list(ast.literal_eval(i).values())[0][0]])

stigmatizingLanguageArray = np.array(stigmatizingLanguage, dtype=object)

# shortens array to 5 elements
stigmatizingLanguageArray = np.array(stigmatizingLanguageArray[0], dtype=object)

stuffToCheck = []
with open(f"/Users/sagewong/Documents/allClinicalNotes/note{stigmatizingLanguageArray[0]}.txt") as file:
    entireFile = file.read()
    fileListSentences = entireFile.replace("\n", "").split(".")
    # list comprehension [sentence, keyvalue[1]] is the output, loops through every sentence in fileListSentences and returns output if keyValue[1] in sentence
    matchingSentence = [sentence for sentence in fileListSentences if stigmatizingLanguageArray[1] in sentence]
    stuffToCheck.append({"sentence with note": matchingSentence, "stigmatizing language": stigmatizingLanguageArray[1], "file path": f"/Users/sagewong/Documents/allClinicalNotes/note{stigmatizingLanguageArray[0]}.txt"})
sentenceWithNote = stuffToCheck[0]['sentence with note']
stigmatizingWord = stuffToCheck[0]['stigmatizing language']
print(f"sentence with note: {sentenceWithNote}")
print(f"stigmatizing language: {stigmatizingWord}")
# print(entireFile)

model = OllamaLLM(model="llama3.2")

result = askLlama(f"replace the stigmatizing word: \"{stigmatizingWord}\" with person-first language in the sentence: \"{sentenceWithNote[0]}\". Return the corrected sentence. Respond only with valid JSON. Do not write an introduction or a summary", sentenceWithNote)
print(f"result: {result}")
print("")

sentence with note: [' Compliant with diuretics but not following low sodium diet or fluid restriction']
stigmatizing language: Compliant
replace the stigmatizing word: "Compliant" with person-first language in the sentence: " Compliant with diuretics but not following low sodium diet or fluid restriction". Return the corrected sentence. Respond only with valid JSON. Do not write an introduction or a summary
{"person": {"condition": "chronic kidney disease", "medication": "diuretics"}}
{'condition': 'chronic kidney disease', 'medication': 'diuretics'}


KeyError: 0